# Utils

In [ ]:
# | default_exp utils


In [ ]:
# | export

from vid_chains.imports import *
import math


/home/rafay/miniconda3/envs/jbt_proj/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# | export


def load_obj_model(name="yolov8n.pt"):
    return YOLO(name)


def detect_objects(model, img):
    res = model(img, stream=True)
    return [{"boxes": r.boxes.data.detach().cpu().tolist()} for r in res]


def get_width(l):
    w = l[2] - l[0]
    return w


def list_widths(obj):
    w = []
    for i in range(0, len(obj.get("boxes"))):
        l = []
        for j in range(0, 6):
            l.append(obj.get("boxes")[i][j])
        if (
            l[5] == 39.0
        ):  # very specific test case for bottles so will ignore other objects, will remove this in the future
            width = get_width(l)
            w.append(width)
    return w


def centroid(l):
    t = []
    cx = (l[0] + l[2]) / 2.0
    cy = (l[1] + l[3]) / 2.0
    t.append(cx)
    t.append(cy)
    return t


def list_centroids(obj):
    c = []
    for i in range(0, len(obj.get("boxes"))):
        l = []
        for j in range(0, 4):
            l.append(obj.get("boxes")[i][j])
        centre = centroid(l)
        c.append(centre)
    return c


def inter_dist(obj):
    c = list_centroids(obj)
    dis = []
    st = []
    for i in range(0, len(c)):
        for j in range(i + 1, len(c)):
            # st.append("Distance b/w object "+str(i)+" and object "+str(j))
            # st.append("D("+str(i)+","+str(j)+")")
            dis.append(math.dist(c[i], c[j]))
    # return st,dis
    return dis


def focal_len_to_px(focal_len, sensor_px):
    return round((focal_len / sensor_px) * 1000)


def camera_to_obj_dist(focal_length_px, obj, real_width):
    widths = list_widths(obj)
    dists = []
    for w in widths:
        distance = (real_width * focal_length_px) / w
        dists.append(distance)

    return dists

In [ ]:
# | hide
# | eval: false

img_path = "../exp_imgs/"

model = load_obj_model()
objects = detect_objects(model, img_path)



image 1/1 /home/rafay/vid_chains/nbs/../exp_imgs/bottle.jpg: 480x640 1 bench, 2 bottles, 212.3ms
Speed: 14.3ms preprocess, 212.3ms inference, 4.9ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
# | hide
# | eval: false


objects


[{'boxes': [[196.56224060058594,
    1210.6712646484375,
    475.10223388671875,
    1623.388916015625,
    0.5954297780990601,
    39.0],
   [468.8839111328125,
    0.817108154296875,
    2632.79833984375,
    926.1826782226562,
    0.46843209862709045,
    13.0],
   [3620.0390625,
    1252.4671630859375,
    3909.64697265625,
    1658.0111083984375,
    0.3052195906639099,
    39.0]]}]

In [ ]:
# | hide
# | eval: false
focal_length_px = focal_len_to_px(6.04, 0.7)  # redmi note 10 pro camera specifications
real_width = 0.0635  # of bottle
cam_dists = [camera_to_obj_dist(focal_length_px, objs, real_width) for objs in objects]
cam_dists

[[1.967191474141819, 1.8920115120625443]]

In [ ]:
# | hide
# | eval: false
dists = [inter_dist(objs) for objs in objects]
dists


[[1544.4955277947797, 3429.2236530738533, 2425.974258601772]]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
